# Scattering Amplitude Reconstruction in Python

###### Giuseppe De Laurentis - University of Edinburgh

[PyHEP indico]("https://indico.cern.ch/event/1566263/timetable/#10-scattering-amplitude-recons")

[mybinder.org/v2/gh/GDeLaurentis/antares/HEAD](https://mybinder.org/v2/gh/GDeLaurentis/antares/HEAD) then go to examples/pyHEP_talk_GDL.ipynb

## Motivation

Next-to-next-to-leading order (NNLO) predictions are essential to match the experimental precision

<img src="1911.00479.crosssection.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">
<center> $\sigma_{pp\rightarrow \gamma\gamma\gamma}$ Chawdhry et al. <a href=https://arxiv.org/abs/1911.00479> arXiv:1911.00479 </a> </center>

without them we cannot tell new physics apart from a mismodelled SM background.

Partonic cross sections read:
$$\small \hat{σ}_{n}=\frac{1}{2\hat{s}}\int d\Pi_{n-2}\;(2π)^4δ^4\big(\sum_{i=1}^n p_i\big)\;|\overline{\mathcal{A}(p_i,h_i,a_i,μ_F, μ_R)}|^2$$

Objective: compute the amplitude at least at NNLO $\mathcal{A} \approx \mathcal{A}^{\text{tree}} + \alpha_s \mathcal{A}^{(1-\text{loop})} + \alpha_s^2 \mathcal{A}^{(2-\text{loop})}$ <br>
Challenges: <br>
1) At NNLO (~ two-loop Feynman diagrams) analytic computations are often unfeasible, especially at high multiplicities
2) Even if an analytic result is avaiable, they may be hard to simplify through analytic manipulations

Simpler results are not just a theorist delight, they are essential for phenomenology

<img src="quad-precision-no-rescue.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

<center> Evaluation of the two-loop corrections to pp -> Vjj obtained in 2021, Abreu et al.<a href=https://arxiv.org/abs/2110.07541> arXiv:2110.07541 </a> </center>

<img src="h2__g_g__Z_b_b.stability.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

<center> Evaluation of the new two-loop amplitude for $pp \to Vjj$, GDL et al.<a href=https://arxiv.org/abs/2503.1059> arXiv:2503.10595 </a> </center>

## What is analytic reconstruction?

A) Compute numerically to bypass all intermediate complexities <br>
B) Analytise the numerical evaluations to infer the exact analytic representation 

<br>

<center> <b> Analytic reconstruction is an alternative to analytic manipulations </b> </center>

<br>

What is it good for:

A) it is needed both to obtain new NNLO corrections <br>
B) and to make available expressions efficient / usable

Efficiency is also a pressing need: <br> the $\alpha_s$ extraction from energy-energy correlators in $pp\rightarrow jjj$ (NNLO, leading color, i.e. up to $1/Nc$ corrections) by ATLAS [arXiv:2301.09351](https://arxiv.org/abs/2301.09351) used $10^8$CPUh

## Why Python over Mathematica?

Among theorists, Mathematica is much more widely used than Python. As a Python user, this is my benefit analysis:

Cons:
1) analytic manipulations are more awkward  (but we will do mostly numerical work)

Pros:
1) Name spaces: variables in Python are local by default, in Mathematica they are global
2) Object oriented: in Python we can attach methods and properties to objects, in Mathematica everything is a glorified list <br>
2b. This makes onboarding a nightmare, you need to know entire code bases to know what methods are avaiable (introspection is great!)
3) Package manager (PyPI), in Mathematica pass files by email or attach to papers (they often break with new versions) <br>
3b. Large community with many available packages
4) Continuous integration: pytest, flint, etc. - in Mathemtica, write a Python script that runs the test
5) Licenses: multi-processing / threading not limited by number of bought licences
6) Open source

Is performance an issue? Not really, outsource compute intensive operations to C++ / CUDA / Fortran / Rust (interfacing is simple)!

## A toy example

Work with spinor-helicity variables:
$$p_{i, \mu} \sigma_\mu^{\dot\alpha\alpha} = p^{\dot\alpha\alpha}_i = \tilde\lambda_i^{\dot\alpha}\lambda_i^{\alpha} = |i]\langle i | \; \text{ if } \;  p^2 = 0 $$
and Mandelstam invariants:
$$s_{ij\dots k} = (p_i + p_j + \dots + p_k)^2 $$

In [1]:
mandelstam_expression = "(1/(⟨14⟩^2⟨15⟩^2⟨23⟩^2))⟨12⟩^3⟨13⟩((4s23(-(s23s34+(s15-s34)s45)^3(s23s34+s45(s15+s34+s45))+s12^3(s15-s23)(s15^3s45+s23^2s34(-s23+s45)+s15^2s45(-s23+s45)+s15(s23^2s34-s23s45^2-s34s45^2))-s12^2(3s15^4s45^2+s15^3s45^2(-4s23-2s34+3s45)+s23s34^2(3s23^3-4s23^2s45+s45^3)+s15^2(-s23s45^2(s34+4s45)-s34s45^2(s34+5s45)+s23^2(s34^2+s45^2))+s15(-4s23^3s34^2+2s34^2s45^3+s23s34s45^2(s34+2s45)+s23^2s45(s34^2+s45^2)))+s12(3s15^4s45^3+s15^3s45^2(4s23s34-2s23s45-4s34s45+3s45^2)+s34^2(s23-s45)^2(3s23^2s34-s34s45^2+s23s45(s34+s45))-s15^2s45(s23^2s34(s34+s45)+s34s45^2(s34+7s45)+2s23s45(2s34^2-s34s45+s45^2))-s15s34(s23-s45)(2s23^2s34(s34-2s45)+s34s45^2(2s34+5s45)+s23s45(2s34^2+2s34s45+s45^2)))))/(3s12^3(s15-s23)s34(s12+s23-s45)s45(s15+s45)(-s12+s34+s45))+(4s23((s23s34+(s15-s34)s45)^2(s23s34+s45(s15+s34+s45))+s12^2(s23^2s34(s23-s45)+s15^3s45+s15^2s45(-s23+s45)-s15(s23^2s34+s23s45^2+s34s45^2))+s12(-2s15^3s45^2+s34^2(-2s23^3+2s23^2s45+s23s45^2-s45^3)+s15^2s45((s34-2s45)s45+s23(-s34+s45))+s15(s23^2s34(s34-s45)+s23s45^3+s34s45^2(s34+3s45))))(-tr5_1234))/(3s12^3(s15-s23)s34(s12+s23-s45)(s12-s34-s45)s45(s15+s45)))[31]"

def black_box_function(phase_space_point):  # may be a complicated expression we want to simplify, or a numerical routine
    return phase_space_point(mandelstam_expression)

In [2]:
# pip install antares-hep (Automated Numerical To Analytical Reconstruction Software) (under development)
from antares import Terms  

In [3]:
# Goal: reconstruct the simplest possible form of this function
oTerms = Terms("""
+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])
+('12354', False, '+')
""")  # this functions is symmetric under 4<->5 exchange, make it manifest

### Flash overview of lips

In [4]:
# pip install lips (Lorentz invariant phase space)
from lips import Particles

In [5]:
# random (complex) phase space point with 5 massless legs; default is complex with 300 digits of precision
# uses mpmath for the arbitrary precision floating-point numbers (mpc and mpf)
oPs = Particles(5)

In [6]:
oPs[1].four_mom # four momentum, p^\mu

array([mpc(real='0.797665907219305524597141209514889165345089658298936999301685171494152458804040891077997689753909086125957224477521681009971621335336947774934242416245013557747661529815896661567009190973713111411632396557558490998250149219650288809978614256956965180658861912023033580847208672599196960230777396504663271', imag='0.719430816873820801517618028228906958437853004742745082082987850201855613329677576047272323663675841374159015991791698996684167385933117453417153494499815943100147350904148454341003576948640329778262452846348204288856839277608975747873799818590411041242360167147762145572115925086860254707275040266533781'),
       mpc(real='-1.39999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999996', imag='-0.47457627118644067796610169

In [7]:
oPs[1].r_sp_d  # right-handed spinor index down

array([[mpc(real='1.22401358812468578886496848243092202810762477202721117176060278098929736665769313579198906728409121265855342221450981899261535919893610863836754773222539175485628071796133963109141947292781528157228435091912397998397421959359558901840080211948089935333903804033818043567783300729753059620822416685814934', imag='0.37358766968205047819718557800926173701041600394698280078635749731095433274951439357618999052192310182074540382833216462636654860404249289482293745857710758455994997789557340639538854510292270525201995005667150577658624984924762600017350460451676315755326042777588728679748450083061108942208846580581615')],
       [mpc(real='-0.0877307433312633304577213996014340426222829459563553533364497750546709851802000902270206241766028048176792510317576449972782297614709679057948511162483951947867361244773267824518325596286625889347623108594786586592713017763761279410167926479682246579984242460858074614996605967990648234148721855636442882', imag='-0.24282641700751353792890

i.e. $\tilde\lambda_{1, \dot\alpha}$

In [8]:
oPs[1].l_sp_d  # left-handed spinor index down

array([[mpc(real='1.22401358812468578886496848243092202810762477202721117176060278098929736665769313579198906728409121265855342221450981899261535919893610863836754773222539175485628071796133963109141947292781528157228435091912397998397421959359558901840080211948089935333903804033818043567783300729753059620822416685814934', imag='0.37358766968205047819718557800926173701041600394698280078635749731095433274951439357618999052192310182074540382833216462636654860404249289482293745857710758455994997789557340639538854510292270525201995005667150577658624984924762600017350460451676315755326042777588728679748450083061108942208846580581615'),
        mpc(real='-2.22139297019482697940015127464900025772284644157293666549230805011700692196970124934758270218204386810148540970578770114677654680253764308720647605608819745288693421128254620640416004487110626495281937595473914711101369537650347173091536472877931911263997997776350696487488022773314502374671500538856698', imag='0.172163479873276504196006768

i.e. $\lambda_{1, \alpha}$

Implements getters and setters, e.g. updating the value of .four_mom will automatically update the spinors (and vice versa)

In [9]:
# I've implemented a custom ast (abstract syntactic tree) parser (this is hidden behind Particles.__call__)
oPs._parse(mandelstam_expression)

"(1/(oPs('⟨1|4⟩')**2*oPs('⟨1|5⟩')**2*oPs('⟨2|3⟩')**2))*oPs('⟨1|2⟩')**3*oPs('⟨1|3⟩')*((4*oPs('s_23')*(-(oPs('s_23')*oPs('s_34')+(oPs('s_15')-oPs('s_34'))*oPs('s_45'))**3*(oPs('s_23')*oPs('s_34')+oPs('s_45')*(oPs('s_15')+oPs('s_34')+oPs('s_45')))+oPs('s_12')**3*(oPs('s_15')-oPs('s_23'))*(oPs('s_15')**3*oPs('s_45')+oPs('s_23')**2*oPs('s_34')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')**2*oPs('s_45')*(-oPs('s_23')+oPs('s_45'))+oPs('s_15')*(oPs('s_23')**2*oPs('s_34')-oPs('s_23')*oPs('s_45')**2-oPs('s_34')*oPs('s_45')**2))-oPs('s_12')**2*(3*oPs('s_15')**4*oPs('s_45')**2+oPs('s_15')**3*oPs('s_45')**2*(-4*oPs('s_23')-2*oPs('s_34')+3*oPs('s_45'))+oPs('s_23')*oPs('s_34')**2*(3*oPs('s_23')**3-4*oPs('s_23')**2*oPs('s_45')+oPs('s_45')**3)+oPs('s_15')**2*(-oPs('s_23')*oPs('s_45')**2*(oPs('s_34')+4*oPs('s_45'))-oPs('s_34')*oPs('s_45')**2*(oPs('s_34')+5*oPs('s_45'))+oPs('s_23')**2*(oPs('s_34')**2+oPs('s_45')**2))+oPs('s_15')*(-4*oPs('s_23')**3*oPs('s_34')**2+2*oPs('s_34')**2*oPs('s_45')**3+oPs('s_23')*oPs(

In [10]:
complex(oTerms(oPs) - oPs(mandelstam_expression)) # evaluation of Terms object is optimized (including upcoming GPU support)

(9.799267994283798e-301-1.8665272370064378e-301j)

In [11]:
expr = "+(8/3s23⟨24⟩[34])/(⟨15⟩⟨34⟩⟨45⟩⟨4|1+5|4])"
complex((oPs(expr) + oPs.image(('12354', False))(expr)) - oTerms(oPs))

(-4.6663180925160944e-302-9.332636185032189e-302j)

In [12]:
# this generates a "real" phase space point (all outgoing convention, energy might be negative)
oPs = Particles(5, real_momenta=True, seed=0)

In [13]:
oPs[1].four_mom

array([mpc(real='0.687332462114479122995963240764965908335348316492734757090320627057009436362388736287994730768383742948855720784042793946320018929144633546574257505225786361677852508913182333102984702295913493771260722480915077025539504603705600363850362366641751820102176413143492357596828095988763161293061866584024905', imag='0.0'),
       mpc(real='-0.010256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256410256411', imag='0.0'),
       mpc(real='0.63636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636363636367',

### Main issue: numerical stability

In [14]:
# pip install syngular (interface and extension to Singular, https://www.singular.uni-kl.de/)
# mainly used for algebraic geometry, but also useful to define the number field
from syngular import Field  
# syntax: Field("field name", characteristic, number of digits)

Simulate a long running computation where error accumulates by lowering the precision

In [15]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=0)

In [16]:
abs(oTerms(oPs) - black_box_function(oPs))  # as expected, the two functions match to 6 digits

mpf('2.38776875e-6')

In [17]:
oPs = Particles(5, field=Field("mpc", 0, 6), seed=2)

In [18]:
abs(oTerms(oPs) - black_box_function(oPs))  # lost precision! :'(

mpf('0.428962946')

In [19]:
black_box_function(oPs)

mpc(real='-1.13071203', imag='25.7996826')

In [20]:
oTerms(oPs)

mpc(real='-0.711602092', imag='25.8910942')

In [21]:
oPs = Particles(5, field=Field("mpc", 0, 16), seed=2)

In [22]:
black_box_function(oPs)

mpc(real='-0.7116312918164890561', imag='25.89107035069941443')

In [23]:
oTerms(oPs)

mpc(real='-0.7116312918068779803', imag='25.89107035076286678')

## finite fields

$$ 
\displaystyle a \in \mathbb{F}_p : a \in \{0, \dots, p -1\} \; \text{ with } \; \{+, -, \times, \div\}
$$

In [24]:
from fractions import Fraction as Q
# pip install pyadic (mainly for p-adic numbers, see later, but also implements finite fields)
# see https://github.com/GDeLaurentis/pyadic
from pyadic import ModP

In [25]:
ModP(Q(1, 2), 11)

6 % 11

or

In [26]:
from syngular import Field

In [27]:
Fp = Field("finite field", 2 ** 31 - 1, 1)
Fp

Field('finite field', 2147483647, 1)

In [28]:
Fp.random()

2134733231 % 2147483647

what goes wrong if p is not prime?

In [29]:
# ModP(Q(1, 2), 6)  # this raises ZeroDivisionError

the inverse cannot be computed for any number not co-prime with the chosen modulus

### Phase space in a finite field

In [30]:
oPs = Particles(5, field=Field("finite field", 2 ** 31 - 1, 1), seed=0)

In [31]:
black_box_function(oPs)

497973027 % 2147483647

In [32]:
oTerms(oPs)

497973027 % 2147483647

these will always match exactly, unless you are unlucky (roughly speaking 1 in $p$ chance) and hit a zero of the denominator, then you'd get a ZeroDivisionError

### The trivial absoulute value

$$ |a|_0 = 0 \; \text{if} \; a = 0 \; \text{else} \; 1$$

In [33]:
abs(ModP(Q(2), 5))

1

In [34]:
abs(ModP(Q(0), 5))

0

### Limitation

a) can make things exactly zero, but not small <br>
b) cannot take limits <br>
c) calculus is undefined

In [35]:
# Fp.epsilon()  # This raises ValueError (no finite field infinitesimal exists)

### [Ostrowski theorem](https://en.wikipedia.org/wiki/Ostrowski%27s_theorem)

There exist 3 possible absolute values on the rationals: <br>
     - the trivial absolute value $|\cdot|_0$ <br>
     - the real absolute value $|\cdot|_\infty$ <br>
     - the p-adic absolute value $|\cdot|_p$

## p-adic numbers

In [36]:
from pyadic import PAdic

In [37]:
ModP(13, 11)

2 % 11

In [38]:
PAdic(13, 11, 3)

2 + 1*11 + O(11^3)

In [39]:
ModP(Q(1, 2), 11)

6 % 11

In [40]:
PAdic(Q(1, 2), 11, 3)  # like for floats, we can have repeating digits (5 here repeats for ever)

6 + 5*11 + 5*11^2 + O(11^3)

In [41]:
# we can divide by p, no more division by zero!

In [42]:
# ModP(Q(1, 11), 11)  # recall this raises ZeroDivisionError: cannot divide by the prime

In [43]:
PAdic(Q(1, 11), 11, 3)

1*11^-1 + O(11^2)

Absolute value goes in descrete steps of powers of $p$ (negative powers are "large", think of $p$ as $\epsilon \ll 1$)

In [44]:
abs(PAdic(Q(1, 11), 11, 3))

O(11^-1)

In [45]:
abs(PAdic(Q(11), 11, 3))

O(11)

In [46]:
abs(PAdic(Q(1, 11), 11, 3)) > abs(PAdic(Q(11), 11, 3))  # I know it's confusing, 11-adically 1/11 is trully larger than 11

True

In [47]:
abs(PAdic(Q(12), 11, 3)) == abs(PAdic(Q(13), 11, 3))  # 11-adically 12 and 13 have the same size (order 11^0)

True

In [48]:
Qp = Field("padic", 2 ** 31 - 1, 11)
Qp

Field('padic', 2147483647, 11)

In [49]:
Qp.random()

159748088 + 981225231*2147483647 + 2019442022*2147483647^2 + 142878177*2147483647^3 + 1231138780*2147483647^4 + 1114892042*2147483647^5 + 1425078810*2147483647^6 + 605566449*2147483647^7 + 1807150731*2147483647^8 + 1498853433*2147483647^9 + 967419695*2147483647^10 + O(2147483647^11)

In [50]:
Qp.epsilon()  # we can do calculs! take limits, compute derivatives, even integrate

2147483647

### Improve Stability

The usual triangle inequality
$$\displaystyle |d(x,z)| \leq |d(x,y)| + |d(y,z)| $$
is strengthened to
$${\displaystyle d(x,z)\leq \max \left\{d(x,y),d(y,z)\right\}}$$

<b> Can never add two numbers and get a larger one than those you started with </b>

### p-adic phase space

In [51]:
oPs = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)

In [52]:
black_box_function(oPs)

1222473774 + 796252591*2147483647 + 1161721493*2147483647^2 + O(2147483647^3)

In [53]:
oTerms(oPs)

1222473774 + 796252591*2147483647 + 1161721493*2147483647^2 + O(2147483647^3)

In [54]:
oPs[1].four_mom  # is None because sqrt(-1) is not in the field

In [55]:
oPs[1].r2_sp  # but the spinors are always in the field (you never need four-momenta to compute Lorentz invariants)

array([[1780157367 + 799679453*2147483647 + 1985151952*2147483647^2 + O(2147483647^3),
        1964726659 + 1313964131*2147483647 + 961046302*2147483647^2 + O(2147483647^3)],
       [1733936412 + 1316287247*2147483647 + 1449001809*2147483647^2 + O(2147483647^3),
        943232507 + 247149017*2147483647 + 1786487518*2147483647^2 + O(2147483647^3)]],
      dtype=object)

In [56]:
oPs = Particles(5, field=Field("padic", 2 ** 31 - 19, 3), seed=0)  # if this is confusing you can pick a different value of p

In [57]:
oPs[1].four_mom

array([42100088 + 1480766398*2147483629 + 1362366842*2147483629^2 + O(2147483629^3),
       1602983086 + 1078739715*2147483629 + 1877805426*2147483629^2 + O(2147483629^3),
       1122307959 + 1972109352*2147483629 + 1326498384*2147483629^2 + O(2147483629^3),
       1918374613 + 112168508*2147483629 + 816814846*2147483629^2 + O(2147483629^3)],
      dtype=object)

In [58]:
# Field("finite field", 2 ** 31 - 1, 1)(1j)  # this raises ValueError, sqrt(-1) is not in the field

In [59]:
Field("finite field", 2 ** 31 - 19, 1)(1j)

629208553 % 2147483629

In [60]:
Field("finite field", 2 ** 31 - 19, 1)(1j) ** 2  == -1

True

## Interpolation algorithms

Peraro [arXiv:1608.01902](https://arxiv.org/abs/1608.01902)

In [61]:
from pyadic.interpolation import Newton_polynomial_interpolation, Thiele_rational_interpolation

In [62]:
def univariate_polynomial(tval):
    return (tval ** 10 + tval - 1)

In [63]:
Newton_polynomial_interpolation(univariate_polynomial, 2 ** 31 - 1, verbose=True)

Finished after 13 samples: [1569849180 % 2147483647, 1078182456 % 2147483647, 1345237137 % 2147483647, 1566393810 % 2147483647, 815006192 % 2147483647, 319562461 % 2147483647, 1293390443 % 2147483647, 677331714 % 2147483647, 380413367 % 2147483647, 1203676396 % 2147483647, 1 % 2147483647, 0 % 2147483647, 0 % 2147483647]. 

t**10 + t - 1

In [64]:
def univariate_rational_func(tval):
    return (tval ** 20 + tval - 1) / (5 - tval)

In [65]:
Thiele_rational_interpolation(univariate_rational_func, 2 ** 31 - 1, verbose=True)

Finished after 40 samples, [584414832 % 2147483647, 970812621 % 2147483647, 1534536413 % 2147483647, 954387064 % 2147483647, 359570102 % 2147483647, 1570313059 % 2147483647, 109498696 % 2147483647, 2123742756 % 2147483647, 1386017183 % 2147483647, 1535674705 % 2147483647, 1521506693 % 2147483647, 1184513925 % 2147483647, 958941582 % 2147483647, 1184542341 % 2147483647, 1008474606 % 2147483647, 1717109627 % 2147483647, 399620187 % 2147483647, 508518201 % 2147483647, 466795340 % 2147483647, 1573390819 % 2147483647, 803901946 % 2147483647, 437215979 % 2147483647, 454478843 % 2147483647, 208954972 % 2147483647, 1633357754 % 2147483647, 1831680191 % 2147483647, 1914540195 % 2147483647, 516669207 % 2147483647, 1330111113 % 2147483647, 1923328681 % 2147483647, 1119820556 % 2147483647, 917973577 % 2147483647, 627695372 % 2147483647, 1365942856 % 2147483647, 789941148 % 2147483647, 1024635069 % 2147483647, 1542141675 % 2147483647, 1565654157 % 2147483647, 376073685 % 2147483647, 507260310 % 214

(t**20 + t - 1)/(5 - t)

### Why Reconstruction $\supset$ Interpolation

As far as I am aware, interpolation algorithms work only for independent sets of variables.

We have variables subject to constraints (e.g. momentum conservation, Schouten identities, etc.)

Interpolation still plays an important role, but it's not the full story

## Least Common Denominator (LCD)

$$ \text{amplitude coeff} = \frac{\mathcal{N}}{\mathcal{D}} = \frac{\mathcal{N}}{\prod_j \mathcal{D}_j^{q_j}}$$

In [66]:
field = Field("finite field", 2 ** 31 - 1, 1)
seed = 0 
oSliceFp = Particles(5, field=field, seed=seed)
oSliceFp.univariate_slice(algorithm='covariant', seed=seed)

In [67]:
import sympy

In [68]:
# momentum conservation is satisfied for all values of the univariate parameter t
sympy.poly(sympy.simplify(oSliceFp.total_mom)[1, 0], modulus=2** 31- 1)

Poly(0, t, modulus=2147483647)

In [69]:
x = sympy.poly(sympy.simplify(oSliceFp("⟨1|4⟩")), modulus=2** 31- 1)
x, x.factor_list()

(Poly(707309386*t + 976617537, t, modulus=2147483647),
 (707309386, [(Poly(t + 525474835, t, modulus=2147483647), 1)]))

In [70]:
from antares.core.settings import settings
from antares.core.numerical_methods import num_func

from lips import Invariants

In [71]:
settings.invariants = Invariants(5).full
print(settings.invariants)  # list of guesses for what the possible singularities of the amplitude can be

['⟨1|2⟩', '[1|2]', '⟨1|3⟩', '[1|3]', '⟨1|4⟩', '[1|4]', '⟨1|5⟩', '[1|5]', '⟨2|3⟩', '[2|3]', '⟨2|4⟩', '[2|4]', '⟨2|5⟩', '[2|5]', '⟨3|4⟩', '[3|4]', '⟨3|5⟩', '[3|5]', '⟨4|5⟩', '[4|5]', '⟨1|(2+3)|1]', '⟨1|(2+5)|1]', '⟨2|(1+3)|2]', '⟨2|(1+5)|2]', '⟨3|(1+2)|3]', '⟨3|(1+5)|3]', '⟨4|(1+2)|4]', '⟨4|(1+5)|4]', '⟨5|(1+2)|5]', '⟨5|(1+4)|5]', '⟨1|(2+3)|(2+5)|1⟩', '[1|(2+3)|(2+5)|1]', '⟨2|(1+3)|(1+5)|2⟩', '[2|(1+3)|(1+5)|2]', '⟨5|(1+2)|(1+4)|5⟩', '[5|(1+2)|(1+4)|5]', 'tr5_1234']


In [72]:
black_box_function.multiplicity = 5
oF = num_func(black_box_function)

In [73]:
oF.get_lcd(oSliceFp, verbose=True)

Finished after 19 samples, [162860843 % 2147483647, 1885044667 % 2147483647, 1411810336 % 2147483647, 1402519729 % 2147483647, 468240741 % 2147483647, 1197585802 % 2147483647, 1947316210 % 2147483647, 291407823 % 2147483647, 450073212 % 2147483647, 882656524 % 2147483647, 1372881572 % 2147483647, 934808415 % 2147483647, 1385619189 % 2147483647, 1938318280 % 2147483647, 1622103845 % 2147483647, 869915779 % 2147483647, 1364220815 % 2147483647, 64322534 % 2147483647, 552291472 % 2147483647]. 
 (-514009628*t**8 + 776677322*t**7 + 991846540*t**6 - 75374069*t**5 + 101225610*t**4 + 36666516*t**3 - 728369354*t**2 + 534567587*t - 70010385)/(t**9 - 49350670*t**8 + 1034569225*t**7 + 606286545*t**6 + 418505031*t**5 - 958181787*t**4 - 882182274*t**3 - 369000092*t**2 - 291896180*t - 698421050)
Polynomial defaultdict(<class 'int'>, {t - 505232661: 1, t - 179967131: 1, t + 594877252: 1, t + 969664725: 1, t**4 + 449954863*t**3 + 382707815*t**2 - 580761694*t - 422746403: 1})
Matched 2 / 8: {'⟨2|3⟩': 1, 

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

In [74]:
oTerms.get_lcd(oSliceFp)

Terms("""+(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")

<img src="variety_slice_v2-transparent.png" style="max-width:100mm; margin-top:2mm; margin-bottom:0mm;">
The blue line is analogous to the univariate slice, but our space has dimension $4n-4=20-4=16$

The planes $\mathcal{D}_i = 0$ are analogous to our denominator factors, but they have dimension $16-1 = 15$

In [75]:
# Univariate interpolation for the LCD is equivalent to checking limits:
field = Field("padic", 2 ** 31 - 1, 5)
seed = 0 
oPs = Particles(5, field=field, seed=seed)
oPs._singular_variety(("⟨1|4⟩", ), (1, ))

In [76]:
oF(oPs)  # simple pole

2107116379*2147483647^-1 + 757787854 + 2019640807*2147483647 + 797457344*2147483647^2 + O(2147483647^3)

In [77]:
oPs._singular_variety(("⟨2|3⟩", ), (1, ))

In [78]:
oF(oPs)  # degree 1 zero

1407121939*2147483647 + 1052185488*2147483647^2 + O(2147483647^3)

Problem: in LCD form the expressions are way to complicated

For instance, for Vjj (at leading color), the largest numerator in LCD form had mass dimension 114, its ansatz size has approx 25M free parameters.

Solution: do partial fraction decompositions <br>
To do this in a multivariate setting we need some algebraic geometry

## Computational Algebraic Geometry \& Multivariate partial fractions

We want to answer this question numerically:
$$ 
\frac{\mathcal{N}}{\mathcal{D}_1\mathcal{D}_2} \stackrel{?}{=}
 \frac{\mathcal{N}_2}{\mathcal{D}_1} + \frac{\mathcal{N}_1}{\mathcal{D}_2} 
$$

<div style="display: flex; margin-top:-6mm;">
    <div style="flex: 1;">
        <img src="V1.png" style="max-width:60%; height:auto;">
    </div>
    <div style="flex: 1; max-width:5%; margin-top:20mm;">
        $\cap$
    </div>
    <div style="flex: 1;">
        <img src="V2.png" style="max-width:60%; height:auto;">
    </div>
    <div style="flex: 1; max-width:5%; margin-top:20mm;">
        $=$
    </div>
    <div style="flex: 1;">
        <img src="V3.png" style="max-width:53%; height:auto;">
    </div>
</div>

$$ 
\color{orange}{\langle xy^2 + y^3 - z^2 \rangle} + \color{blue}{\langle x^3 + y^3 - z^2 \rangle} = \langle xy^2 + y^3 - z^2, x^3 + y^3 - z^2 \rangle = \color{red}{\langle 2y^3-z^2, x-y \rangle} \cap \color{green}{\langle y^3-z^2, x \rangle} \cap \color{purple}{\langle z^2, x+y \rangle}
$$

In [79]:
from syngular import Ring, Ideal  # Requires Singular (e.g. apt install singular)

In [80]:
ring = Ring('0', ('x', 'y', 'z'), 'dp')

In [81]:
I = Ideal(ring, ['x*y^2+y^3-z^2', ])
J = Ideal(ring, ['x^3+y^3-z^2', ]) 

In [82]:
K1 = Ideal(ring, ['2*y^3 - z^2', 'x-y', ])
K2 = Ideal(ring, ['y^3-z^2', 'x', ])
K3 = Ideal(ring, ['z^2', 'x+y', ])

In [83]:
I + J == K1 & K2 & K3

True

### Points on varieties (finding solutions to arbiraty systems of equations)

In [84]:
from syngular import RingPoint

In [85]:
field = Field("finite field", 2 ** 31 - 1, 1)

In [86]:
point = RingPoint(ring, field)
point.update(K1.point_on_variety(field=field))

In [87]:
[point(generator) for generator in K1.generators]

[0 % 2147483647, 0 % 2147483647]

In [88]:
[point(generator) for generator in K2.generators]

[1817970875 % 2147483647, 412685906 % 2147483647]

In [89]:
field = Field("padic", 2 ** 31 - 1, 3)

In [90]:
point = RingPoint(ring, field)
point.update(K1.point_on_variety(field=field, valuations=(1, 1)))

In [91]:
[point(generator) for generator in K1.generators]

[718372410*2147483647 + 1296125827*2147483647^2 + O(2147483647^3),
 1464285392*2147483647 + 2046458951*2147483647^2 + O(2147483647^3)]

In [92]:
[point(generator) for generator in K2.generators]

[315675511 + 443827638*2147483647 + 1147775447*2147483647^2 + O(2147483647^3),
 679487827 + 1899488474*2147483647 + 1473546938*2147483647^2 + O(2147483647^3)]

The partial fraction decomposition is valid, if the numerator vanishes on all branches (K1, K2, K3 here)

## The geometry of phase space

In [93]:
from lips.algebraic_geometry.covariant_ideal import LipsIdeal

In [94]:
oPs = Particles(5)
oPs.make_analytical_d()

In [95]:
oPs["|1⟩"], oPs["[1|"]

(array([[a1],
        [b1]], dtype=object),
 array([[c1, d1]], dtype=object))

In [96]:
LipsIdeal.__bases__

(syngular.ideal.Ideal,)

### Geometry of singular phase space

In [97]:
oPs = Particles(5, field=Field("padic", 2 ** 31 - 1, 5))

In [98]:
J = LipsIdeal(5, ("⟨4|1+5|4]", "⟨5|1+4|5]", ))
J

Ideal over Ring
    0, (a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, a4, b4, c4, d4, a5, b5, c5, d5), dp
generated by
    4*a1*b4*c1*d4 - 4*a1*b4*c4*d1 - 4*a4*b1*c1*d4 + 4*a4*b1*c4*d1 + 4*a4*b5*c4*d5 - 4*a4*b5*c5*d4 - 4*a5*b4*c4*d5 + 4*a5*b4*c5*d4,4*a1*b5*c1*d5 - 4*a1*b5*c5*d1 + 4*a4*b5*c4*d5 - 4*a4*b5*c5*d4 - 4*a5*b1*c1*d5 + 4*a5*b1*c5*d1 - 4*a5*b4*c4*d5 + 4*a5*b4*c5*d4,b1*d1 + b2*d2 + b3*d3 + b4*d4 + b5*d5,-a1*d1 - a2*d2 - a3*d3 - a4*d4 - a5*d5,-b1*c1 - b2*c2 - b3*c3 - b4*c4 - b5*c5,a1*c1 + a2*c2 + a3*c3 + a4*c4 + a5*c5

In [99]:
J.dim, J.codim

(14, 6)

The following line checks whether this ideal is prime (it isn't)

In [100]:
J.test_primality(verbose=True)

gathering f-poly factors: @ 37/38 of which 0 timedout                                   
easiest projection is 208: (1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1) of degree 1
The ideal is radical: True
 smallest f poly factors (33): complexity 856                              
Original ideal has codim 6
 at factor 0: 1.                                       
 at factor 1: d2.                                       
 at factor 2: a4.                                       
 at factor 3: a1.                                       
 at factor 4: a3.                                       
 at factor 5: a5.                                       
 at factor 6: b1.                                       
 at factor 7: d4.                                       
 at factor 8: b4.                                       
 at factor 9: d3.                                       
 at factor 10: d5.                                       
 at factor 11: c2.                                    

False

We need the following 3 prime ideals

In [101]:
K = LipsIdeal(5, ("⟨14⟩", "⟨15⟩", "⟨45⟩", "[23]"))
L = LipsIdeal(5, ("⟨12⟩", "⟨13⟩", "⟨14⟩", "⟨15⟩", "⟨23⟩", "⟨24⟩", "⟨25⟩", "⟨34⟩", "⟨35⟩", "⟨45⟩"))
M = LipsIdeal(5, ("⟨4|1+5|4]", "⟨5|1+4|5]", "|1]⟨14⟩⟨15⟩+|4]⟨14⟩⟨45⟩-|5]⟨45⟩⟨15⟩", "|1⟩[14][15]+|4⟩[14][45]-|5⟩[45][15]"))

The following verifies they are indeed prime

In [102]:
assert K.test_primality() and L.test_primality() and M.test_primality()

\& operator means intersection ($\cap$), like for sets. The following checks that the ideal J is indeed an intersection of 5 prime ideals.

In [103]:
assert K & K("12345", True) & L & L("12345", True) & M == J

In other words, the variety (= hyper-surface) V(J) is the union of V(K), V(K-bar), V(L), V(L-bar) and V(M)

### Phase-space points on irreducible varieties

We can now use this to find if a partial fraction decomposition is possible, using numerics only. <br>
Normally, we have access to a "black box function" for the rational expression, and the common denominator.

In [104]:
common_denominator = "(⟨14⟩⟨15⟩⟨34⟩⟨35⟩⟨45⟩⟨4|1+5|4]⟨5|1+4|5])"

In [105]:
oPsK = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsK._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=K.generators)

In [106]:
black_box_function(oPsK) * oPsK(common_denominator)  # rational * denominator is a proxy for the numerator polynomial

679273289*2147483647^3 + 1895573676*2147483647^4 + O(2147483647^5)

In [107]:
oPsKb = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsKb._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=K("12345", True).generators)

In [108]:
black_box_function(oPsKb) * oPsKb(common_denominator)

1174139383*2147483647^2 + 2092180996*2147483647^3 + O(2147483647^4)

In [109]:
oPsL = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsL._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=L.generators)

In [110]:
black_box_function(oPsL) * oPsL(common_denominator)

1990154823*2147483647^5 + O(2147483647^6)

In [111]:
oPsLb = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsLb._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=L("12345", True).generators)

In [112]:
black_box_function(oPsL) * oPsL(common_denominator)

1990154823*2147483647^5 + O(2147483647^6)

In [113]:
oPsM = Particles(5, field=Field("padic", 2 ** 31 - 1, 3), seed=0)
oPsM._singular_variety(("⟨4|1+5|4]", "⟨5|1+4|5]"), (1, 1), generators=M.generators)

In [114]:
black_box_function(oPsM) * oPsM(common_denominator)

318569542*2147483647 + 1398042680*2147483647^2 + O(2147483647^3)

The numerator vanishes on all sub-varities => the decomposition is valid

## Outlook (currently private / under development)

In [115]:
oTermsAnsatzLCD = Terms("""
+(⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])
""")
oTermsAnsatzLCD.oUnknown = oF
oTermsAnsatzLCD.as_ansatz()

Obtaining ansatz from CP_SAT with lM, lPW: [6.0], [[1, 1, 0, 1, 1]]. Generating ansatz for: 6.0, [1, 1, 0, 1, 1]Status = OPTIMAL
Number of solutions found: 30
Obtained ansatz from CP_SAT with lM, lPW: [6.0], [[1, 1, 0, 1, 1]]. Size: 30.                                                       


In [116]:
import syngular
from syngular import TemporarySetting

with TemporarySetting(syngular, "USE_ELLIPSIS_FOR_PRINT", True):
    print(oTermsAnsatzLCD)  # 30 free parameters

+⟨2|3⟩[2|3](?⟨1|4⟩⟨2|4⟩²⟨2|5⟩[2|4]²+...⟪28 terms⟫...+?⟨1|2⟩⟨4|5⟩³[4|5]²)/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|1+5|4]⟨5|1+4|5])


In [117]:
oTermsAnsatz = Terms("""
+(⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4])
+(⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨5|(1+4)|5])
""")
oTermsAnsatz.oUnknown = oF
oTermsAnsatz.as_ansatz()

Obtaining ansatz from CP_SAT with lM, lPW: [4.0, 4.0], [[1, 1, 0, 1, 1], [1, 1, 0, 1, 1]]. Generating ansatz for: 4.0, [1, 1, 0, 1, 1]Status = OPTIMAL
Number of solutions found: 10
Generating ansatz for: 4.0, [1, 1, 0, 1, 1]Status = OPTIMAL
Number of solutions found: 10
Obtained ansatz from CP_SAT with lM, lPW: [4.0, 4.0], [[1, 1, 0, 1, 1], [1, 1, 0, 1, 1]]. Size: 20.                                                       


In [118]:
with TemporarySetting(syngular, "USE_ELLIPSIS_FOR_PRINT", True):
    print(oTermsAnsatz) # less than 20 free parameters (redundancies among the two fractions are not yet removed)

+⟨2|3⟩[2|3](?⟨1|4⟩⟨2|4⟩⟨2|5⟩[2|4]+...⟪8 terms⟫...+?⟨1|2⟩⟨4|5⟩²[4|5])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|1+5|4])
+⟨2|3⟩[2|3](?⟨1|4⟩⟨2|4⟩⟨2|5⟩[2|4]+...⟪8 terms⟫...+?⟨1|2⟩⟨4|5⟩²[4|5])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨5|1+4|5])


In general the gain from partial fractioning is significant, e.g. from millions to tens of thousands free parameters <br>
Up next: solving the linear system

(py)CUDA finite field solver coming soon: Linac (Linear Algebra with CUDA)

<img src="cubic_fit.png" style="max-width:150mm; margin-top:2mm; margin-bottom:0mm;">

Work in collaboration with Jack Franklin (recently graduated PhD from Durham, now at Cambridge) <br>

We have tested this on systems of size up to 100k (which takes only 45 minutes on an A100!)

In [119]:
import mpmath
mpmath.mp.dps = 300 

In [120]:
from antares.core.unknown import Unknown
from antares.numerical_to_analytical import numerical_to_analytical

In [121]:
oF.do_single_collinear_limits()
oF.do_double_collinear_limits()

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0, 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0, 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0, 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0, 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0, 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0, 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨1|5⟩, ⟨4|(1+5)|4]]:       2.0, 3  → 2
[⟨1|5⟩, ⟨5|(1+4)|5]]:       1.0, 3  → 2
[⟨3|4⟩, ⟨3|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|5⟩]:             3.0, 31 → 7
[⟨3|4⟩, ⟨4|(1+5)|4]]:

In [122]:
oU = Unknown(oF)

In [123]:
numerical_to_analytical(oU)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
NoneToAnalytical called at depth 0.

The partial result is:                                                                                             
(1⟨2|3⟩[2|3])/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])

Mass dimension & phase weights: -1.0, [-1, 1, -2, -2, -2] → 6.0, [1, 1, 0, 1, 1]
Cleaning pair scalings results from known numerator information.                                                             
Finished calculating pair scalings. They are:                         
[⟨1|4⟩, ⟨1|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨3|4⟩]:             3.0 (3.0), 26 → 7
[⟨1|4⟩, ⟨3|5⟩]:             2.0 (2.0), 2  → 2
[⟨1|4⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|4⟩, ⟨4|(1+5)|4]]:       1.0 (1.0), 3  → 2
[⟨1|4⟩, ⟨5|(1+4)|5]]:       2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|4⟩]:             2.0 (2.0), 4  → 2
[⟨1|5⟩, ⟨3|5⟩]:             3.0 (3.0), 26 → 7
[⟨1|5⟩, ⟨4|5⟩]:             3.0 (3.0), 31 → 7
[⟨1|

[Terms("""+[2|3]⟨2|3⟩(8/3⟨3|5⟩⟨2|5⟩[2|3]⟨1|4⟩[4|5]⟨2|4⟩-8/3⟨2|5⟩⟨1|3⟩⟨2|4⟩⟨4|5⟩[3|5][2|4]-16/3⟨3|5⟩[3|4]⟨1|3⟩⟨2|4⟩⟨4|5⟩[3|5]+8/3⟨3|5⟩[3|4]⟨3|4⟩[3|5]⟨4|5⟩⟨1|2⟩+8/3⟨2|3⟩[3|4]⟨1|3⟩[3|5]⟨4|5⟩²)/(⟨1|4⟩⟨1|5⟩⟨3|4⟩⟨3|5⟩⟨4|5⟩⟨4|(1+5)|4]⟨5|(1+4)|5])""")]

## GitHub self hosted runner (for GPU access)

Something you might find useful: [github.com/GDeLaurentis/docker-gpu-runner-for-github-actions](https://github.com/GDeLaurentis/docker-gpu-runner-for-github-actions)